# Invalid OSA for Nestle-Cereals Tesco

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import datetime as dtm

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style

import warnings
warnings.filterwarnings('ignore')
style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20, 8)

### Load data

In [3]:
df_alerts = pd.read_csv('/anfvol1/Tara-Notebooks/data/nestle_cereals_tesco_2019.01.27_nestlecereals_drt_uk_alerts.csv',
                parse_dates=['DateGenerated', 'LastDateIncluded'])
selected_cols_alert = ['ChainRefExternal', 'RefExternal', 'Issue', 'LostSalesValue']
df_alerts = df_alerts[selected_cols_alert]

df_alerts = df_alerts.rename({'LostSalesValue': 'LSV', 'ChainRefExternal':'organization_unit_num', 'RefExternal':'retailer_item_id'}, axis='columns')
df_alerts.head()

,organization_unit_num,retailer_item_id,Issue,LSV
0,5014,73445497,NaN,72.04
1,5014,50209069,NaN,72.04
2,6255,73445497,NaN,138.16
3,6255,50209069,NaN,138.16
4,5030,73445497,NaN,46.80


In [4]:
df = pd.read_csv('/anfvol1/Tara-Notebooks/data/nestle_cereals_tesco_2019.01.27_nestlecereals_drt_uk_alerts_historicalData.csv',
                parse_dates=['SALES_DT'])
df = df.rename(str.lower, axis='columns')
df.loc[:, 'drfe_pos_item_qty'] = df['drfe_pos_item_qty'].clip(0, 100000)
df = df.rename({'drfe_pos_item_qty':'expected_pos'}, axis='columns')
df.head()

,organization_unit_num,retailer_item_id,retailer_item_desc,retailer,client,country_code,sales_dt,pos_item_qty,pos_amt,on_hand_inventory_qty,expected_pos,drfe_error,drfe_load_ts
0,2278,84643545,NESTLE COOKIE CRISP CEREAL 500G,TESCO,NESTLECEREALS,UK,2019-12-14,3.0,6.00,9.0,1.62,-1.38,2020-01-02T22:03:31.801Z
1,2278,84643545,NESTLE COOKIE CRISP CEREAL 500G,TESCO,NESTLECEREALS,UK,2019-03-01,0.0,0.00,11.0,1.34,1.34,2019-10-25T03:00:05.208Z
2,2278,84643545,NESTLE COOKIE CRISP CEREAL 500G,TESCO,NESTLECEREALS,UK,2019-10-23,9.0,12.78,17.0,2.23,-6.77,2020-01-02T22:03:31.801Z
3,2278,84643545,NESTLE COOKIE CRISP CEREAL 500G,TESCO,NESTLECEREALS,UK,2019-05-08,3.0,6.00,15.0,3.06,0.06,2019-10-25T03:00:05.208Z
4,2278,84643545,NESTLE COOKIE CRISP CEREAL 500G,TESCO,NESTLECEREALS,UK,2019-08-04,0.0,0.00,22.0,0.80,0.80,2019-10-25T03:00:05.208Z


### Generate OSA alerts

In [5]:
def lsv(df, only_positive_error='True'):
    df.loc[:, 'error'] = df['expected_pos'] - df['pos_item_qty']
    
    if only_positive_error=='True':
        df.loc[:, 'error'][df['error']<0] = 0
        
    df = df.sort_values(['retailer_item_id', 'organization_unit_num', 'sales_dt'])
    avg_error = df.groupby(['retailer_item_id', 'organization_unit_num'], as_index=False)['error'].rolling(2).mean().shift(1)
    avg_price = df.groupby(['retailer_item_id', 'organization_unit_num'], as_index=False)['price'].rolling(2).mean().shift(1)
    lsv_df = avg_error * avg_price
    
    df = df.join(pd.DataFrame(data=lsv_df.values, columns=['LSV'], index=[x[1] for x in lsv_df.index]))
    df = df.join(pd.DataFrame(data=avg_error.values, columns=['Alert'], index=[x[1] for x in avg_error.index]))
    
    df.loc[:, 'LSV'][df['LSV']<=0]=np.nan
    df.loc[:, 'Alert'][df['Alert']<=0]=np.nan
    return df

### Invalid osa

In [6]:
def add_daily_group(df, col_name='expected_pos', percentile_list=[0.8, 0.6], minimum_pos=3):
    """
    Assigns each store/day to one of high-, middle-, slow-selling group
    
    param df: input dataframe for one item
    param string col_name: choose beteween 'pos_item_qty' or 'expected_pos'
    param list percentile_list: [high percentile, low percentile]
    param int minimum_pos: minimum pos to define a slow selling item
    """
    
    percentile_high, percentile_low = percentile_list
    top_threshold_qty = df[col_name].quantile(percentile_high)
    middle_threshold_qty = df[col_name].quantile(percentile_low)
    
    df['group'] = df.shape[0] * np.nan
    if middle_threshold_qty <3:
        df.loc[:, 'group'] = 'slow'
    else:
        df.loc[:, 'group'][(df[col_name]>middle_threshold_qty) & (df[col_name]<=top_threshold_qty)] = 'middle'
        df.loc[:, 'group'][df[col_name]>top_threshold_qty] = 'top'
        df.loc[:, 'group'][df[col_name]<=middle_threshold_qty] = 'slow'
    
    return df

def select_percentile_comb(df, percentile=0.8):
    """
    Calculates how often a item/store aprears in each group. In the list of all appeared days,
    calculated x-percentile is the threshold to select stores.
    param datafram df: Input dataframe with a column group
    param float percentile: percentile to calculate the infimum number of days
    return dataframe: dataframe which contains valid stores for each retailer_item_id and group
    """
    grouped_days = df.groupby(['retailer_item_id', 'organization_unit_num', 'group'], 
                                as_index=False)['sales_dt'].count()\
                        .rename({'sales_dt': '# days in the group'}, axis='columns')
    grouped_stores = grouped_days.groupby(['retailer_item_id', 'group'], 
                                          as_index=False)['# days in the group'].quantile(.8)\
                        .rename({'# days in the group': 'membership_threshold'}, axis='columns')
    
    grouped_days = grouped_days.merge(grouped_stores)
    
    grouped_days = grouped_days[grouped_days['# days in the group']>=grouped_days['membership_threshold']]
    
    return grouped_days

In [22]:
def invalid_osa(df, valid_osa_ratio):
    print(df.columns)
    total_store_num = df.groupby(['retailer_item_id'], as_index=False)['organization_unit_num'].count().rename({'organization_unit_num': 'total_store_num'}, axis='columns')

    df = df[df['LSV']>0][['retailer_item_id', 'organization_unit_num', 'sales_dt']]
    
    days_store_num = df.groupby(['retailer_item_id', 'sales_dt'], as_index=False)['organization_unit_num'].count().rename({'organization_unit_num': 'count_stores'}, axis='columns')
    days_store_num = days_store_num.merge(total_store_num)
    days_store_num.loc[:, 'ratio_stores'] = days_store_num['count_stores']/days_store_num['total_store_num']
    
    return days_store_num[days_store_num['ratio_stores']>=valid_osa_ratio]

In [8]:
def build_alert_table(df, col_name, membership_multiplier):
    df_with_q3 = df.groupby(['retailer_item_id', 'group', 'sales_dt'], as_index=False)[col_name].quantile(0.75).rename({col_name: 'Q3'}, axis='columns')
    df_with_q1 = df.groupby(['retailer_item_id', 'group', 'sales_dt'], as_index=False)[col_name].quantile(0.25).rename({col_name: 'Q1'}, axis='columns')
    df_with_q1_q3 = df_with_q3.merge(df_with_q1)
    df_with_q1_q3.loc[:, 'outlier threshold'] = df_with_q1_q3['Q1'] - membership_multiplier * (df_with_q1_q3['Q3'] - df_with_q1_q3['Q1'])

    df = df.merge(df_with_q1_q3)
    
    df_low_sales_alert= df[(df[col_name]<= df['outlier threshold'])]
    df_low_sales_alert.loc[:, 'LSV Low Sales'] = (df_low_sales_alert['Q1'] - df_low_sales_alert[col_name]) * df_low_sales_alert['price']
    return df_low_sales_alert

### Analysis

In [9]:
selected_cols = ['sales_dt', 'organization_unit_num', 'retailer_item_id', 'pos_item_qty', 'pos_amt', 'expected_pos']

print(df['sales_dt'].max())

start_date = '2019-09-01'
end_date = '2020-01-20'
minimum_day_ratio = 0.3

df_inp = df[selected_cols][(df['sales_dt']>=start_date) & (df['sales_dt']<=end_date)]
df_inp.head()

2020-01-26 00:00:00


,sales_dt,organization_unit_num,retailer_item_id,pos_item_qty,pos_amt,expected_pos
0,2019-12-14,2278,84643545,3.0,6.00,1.62
2,2019-10-23,2278,84643545,9.0,12.78,2.23
5,2019-09-26,2278,84643545,0.0,0.00,0.85
6,2019-12-20,2278,84643545,4.0,8.00,3.63
10,2019-11-24,2278,84643545,2.0,5.70,1.33


In [10]:
df_grouped = df_inp.groupby(['retailer_item_id']).apply(add_daily_group)

print(df_inp.shape)
# drop all days in group 'slow'
df_grouped = df_grouped[df_grouped['group']!='slow']

membership_threshold_df = select_percentile_comb(df_grouped, 0.8)

# Decide to accept threshold
minimum_days = (dtm.datetime.strptime(end_date, '%Y-%m-%d') - dtm.datetime.strptime(start_date, '%Y-%m-%d')).days
membership_threshold_df = membership_threshold_df[membership_threshold_df['membership_threshold']>=minimum_day_ratio*minimum_days]

df_out = df_inp.merge(membership_threshold_df)
print(df_out.shape)
df_out.head()

(1257440, 6)
(20868, 9)


,sales_dt,organization_unit_num,retailer_item_id,pos_item_qty,pos_amt,expected_pos,group,# days in the group,membership_threshold
0,2019-09-18,6785,52162304,38.0,63.0,27.40,top,50,46.0
1,2019-10-07,6785,52162304,40.0,66.0,0.02,top,50,46.0
2,2019-11-20,6785,52162304,3.0,9.9,6.53,top,50,46.0
3,2019-11-29,6785,52162304,3.0,9.9,5.49,top,50,46.0
4,2019-12-19,6785,52162304,3.0,9.9,3.80,top,50,46.0


In [11]:
assigned_grouped = df_out.groupby(['organization_unit_num', 'retailer_item_id', 'group'], as_index=False)['organization_unit_num', 'retailer_item_id', 'group'].apply(lambda x: x.drop_duplicates())
assigned_grouped.head()

,,organization_unit_num,retailer_item_id,group
0,19035,2007,52162304,top
1,5499,2008,52162304,top
2,6768,2019,52162304,top
3,4512,2020,52162304,top
4,8319,2025,52162304,top


In [12]:
df_lastDay = df[df['sales_dt']=='2020-01-26']
df_lastDay.loc[:, 'price'] = df_lastDay['pos_amt']/df_lastDay['pos_item_qty']
df_lastDay = df_lastDay.merge(assigned_grouped, how='inner')

In [13]:
df_lastDay.head()

,organization_unit_num,retailer_item_id,retailer_item_desc,retailer,client,country_code,sales_dt,pos_item_qty,pos_amt,on_hand_inventory_qty,expected_pos,drfe_error,drfe_load_ts,price,group
0,6785,52162304,NESTLE CHEERIOS MULTIGRAIN CEREAL 600G,TESCO,NESTLECEREALS,UK,2020-01-26,3.0,9.90,68.0,7.15,4.15,2020-01-24T12:27:03.857Z,3.30,top
1,3031,52162304,NESTLE CHEERIOS MULTIGRAIN CEREAL 600G,TESCO,NESTLECEREALS,UK,2020-01-26,7.0,23.10,25.0,9.65,2.65,2020-01-24T12:27:03.857Z,3.30,top
2,2328,52162304,NESTLE CHEERIOS MULTIGRAIN CEREAL 600G,TESCO,NESTLECEREALS,UK,2020-01-26,4.0,13.20,108.0,7.51,3.51,2020-01-24T12:27:03.857Z,3.30,top
3,5851,52162304,NESTLE CHEERIOS MULTIGRAIN CEREAL 600G,TESCO,NESTLECEREALS,UK,2020-01-26,2.0,6.60,67.0,6.06,4.06,2020-01-24T12:27:03.857Z,3.30,top
4,3297,52162304,NESTLE CHEERIOS MULTIGRAIN CEREAL 600G,TESCO,NESTLECEREALS,UK,2020-01-26,3.0,9.57,29.0,5.81,2.81,2020-01-24T12:27:03.857Z,3.19,top


In [14]:
membership_multiplier = 1
df_low_sales_alert = build_alert_table(df_lastDay, col_name='expected_pos', membership_multiplier=membership_multiplier)

In [15]:
df_low_sales_alert.head()

,organization_unit_num,retailer_item_id,retailer_item_desc,retailer,client,country_code,sales_dt,pos_item_qty,pos_amt,on_hand_inventory_qty,expected_pos,drfe_error,drfe_load_ts,price,group,Q3,Q1,outlier threshold,LSV Low Sales


In [16]:
df_invalid_alerts = df_lastDay.merge(df_alerts, how='left')
df_invalid_alerts.head()

,organization_unit_num,retailer_item_id,retailer_item_desc,retailer,client,country_code,sales_dt,pos_item_qty,pos_amt,on_hand_inventory_qty,expected_pos,drfe_error,drfe_load_ts,price,group,Issue,LSV
0,6785,52162304,NESTLE CHEERIOS MULTIGRAIN CEREAL 600G,TESCO,NESTLECEREALS,UK,2020-01-26,3.0,9.90,68.0,7.15,4.15,2020-01-24T12:27:03.857Z,3.30,top,NaN,NaN
1,3031,52162304,NESTLE CHEERIOS MULTIGRAIN CEREAL 600G,TESCO,NESTLECEREALS,UK,2020-01-26,7.0,23.10,25.0,9.65,2.65,2020-01-24T12:27:03.857Z,3.30,top,NaN,NaN
2,2328,52162304,NESTLE CHEERIOS MULTIGRAIN CEREAL 600G,TESCO,NESTLECEREALS,UK,2020-01-26,4.0,13.20,108.0,7.51,3.51,2020-01-24T12:27:03.857Z,3.30,top,NaN,NaN
3,5851,52162304,NESTLE CHEERIOS MULTIGRAIN CEREAL 600G,TESCO,NESTLECEREALS,UK,2020-01-26,2.0,6.60,67.0,6.06,4.06,2020-01-24T12:27:03.857Z,3.30,top,NaN,NaN
4,3297,52162304,NESTLE CHEERIOS MULTIGRAIN CEREAL 600G,TESCO,NESTLECEREALS,UK,2020-01-26,3.0,9.57,29.0,5.81,2.81,2020-01-24T12:27:03.857Z,3.19,top,NaN,NaN


In [23]:
invalid_osa(df_invalid_alerts, 0.7)

Index(['organization_unit_num', 'retailer_item_id', 'retailer_item_desc',
       'retailer', 'client', 'country_code', 'sales_dt', 'pos_item_qty',
       'pos_amt', 'on_hand_inventory_qty', 'expected_pos', 'drfe_error',
       'drfe_load_ts', 'price', 'group', 'Issue', 'LSV'],
      dtype='object')


,retailer_item_id,sales_dt,count_stores,total_store_num,ratio_stores


### Plots